<h1 align="center" style="color:blue"> AtliQs Transaction Data Preprocessing  </h1>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

<h2 align="center" style="color:blue">Data Loading </h2>

In [11]:
fae = pd.read_csv("FactActualsEstimates.csv")
fae.shape

(1858329, 11)

In [13]:
fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty'],
      dtype='object')

In [21]:
fae.head()

,date,division,category,product_code,product,market,platform,channel,customer_code,customer_name,qty,fiscal_year
0,2017-09-01,N & S,External Solid State Drives,A6218160101,AQ Digit SSD,Australia,Brick & Mortar,Direct,70008169,AltiQ Exclusive,81,2018
1,2017-09-01,N & S,External Solid State Drives,A6218160101,AQ Digit SSD,Australia,Brick & Mortar,Retailer,90008165,Forward Stores,157,2018
2,2017-09-01,N & S,External Solid State Drives,A6218160101,AQ Digit SSD,Australia,Brick & Mortar,Retailer,90008166,Sound,126,2018
3,2017-09-01,N & S,External Solid State Drives,A6218160101,AQ Digit SSD,Australia,Brick & Mortar,Retailer,90008167,Electricalsocity,160,2018
4,2017-09-01,N & S,External Solid State Drives,A6218160101,AQ Digit SSD,Australia,E-Commerce,Direct,70008170,Atliq e Store,120,2018


<h2 align="center" style="color:blue"> Fiscal Year column </h2>

since gross price table has fiscal year, hence we are deriving a fiscal year column from fae["date"] by adding 4 month to each date

In [15]:
fae.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858329 entries, 0 to 1858328
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   division       object
 2   category       object
 3   product_code   object
 4   product        object
 5   market         object
 6   platform       object
 7   channel        object
 8   customer_code  int64 
 9   customer_name  object
 10  qty            int64 
dtypes: int64(2), object(9)
memory usage: 156.0+ MB


In [17]:
fae['date'] = pd.to_datetime(fae['date'])

In [19]:
fae['fiscal_year'] = (fae['date'] + pd.DateOffset(months=4)).dt.year

<h2 align="center" style="color:blue">Importing and joining with gross price table </h2>

We need gross price table to calculate gross sales amount in transactions table

In [26]:
gp = pd.read_csv("gross_df.csv")
gp.shape

(1197, 3)

In [28]:
gp.columns

Index(['product_code', 'fiscal_year', 'gross_price'], dtype='object')

## Join fae with gp table on product_code, fiscal_year

In [30]:
merged_fae = pd.merge(
    fae,
    gp,
    how='left',  # or 'inner', 'right', 'outer' depending on your use case
    on=['product_code', 'fiscal_year']
)

In [32]:
merged_fae.shape

(1858329, 13)

In [34]:
merged_fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty',
       'fiscal_year', 'gross_price'],
      dtype='object')

<h2 align="center" style="color:blue">Gross Sales Amount</h2>

gross price is the price of single unit of that respective product_code in that fiscal_year
we need to calculate gross_price_amt which is qty * gross_price

In [36]:
merged_fae["gross_sales_amt"] = merged_fae["gross_price"] * merged_fae["qty"]

In [38]:
merged_fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty',
       'fiscal_year', 'gross_price', 'gross_sales_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Importing and joining with pre_invoice_deductions table </h2>

Importing pre invoice table is important to get pre inovice discount pct to calculate pre invoice deductions

#### Importing pre_invoice_deductions table from pre_df.csv

In [42]:
pre = pd.read_csv("pre_df.csv")
pre.shape

(1045, 3)

In [44]:
pre.columns

Index(['customer_code', 'fiscal_year', 'pre_invoice_discount_pct'], dtype='object')

## Join merged_fae with pre table on customer_code, fiscal_year

In [47]:
nmerge_fae = pd.merge(
    merged_fae,
    pre,
    how='left',  # or 'inner', 'right', 'outer' depending on your use case
    on=['customer_code', 'fiscal_year']
)

In [49]:
nmerge_fae.shape

(1858329, 15)

In [51]:
nmerge_fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty',
       'fiscal_year', 'gross_price', 'gross_sales_amt',
       'pre_invoice_discount_pct'],
      dtype='object')

<h2 align="center" style="color:blue">Calculating pre_invoice_disc_amt column </h2>

In [29]:
nmerge_fae["pre_invoice_disc_amt"] = nmerge_fae["gross_sales_amt"] * nmerge_fae["pre_invoice_discount_pct"]

In [30]:
nmerge_fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty',
       'fiscal_year', 'gross_price', 'gross_sales_amt',
       'pre_invoice_discount_pct', 'pre_invoice_disc_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Calculating net_invoice_sales_amt  </h2>

net_invoice_sales_amt = gross_sales amount - pre_invoice_disc_amt column

In [32]:
nmerge_fae["net_invoice_sales_amt"] = nmerge_fae["gross_sales_amt"] - nmerge_fae["pre_invoice_disc_amt"]

In [61]:
nmerge_fae.columns

Index(['date', 'division', 'category', 'product_code', 'product', 'market',
       'platform', 'channel', 'customer_code', 'customer_name', 'qty',
       'fiscal_year', 'gross_price', 'gross_sales_amt',
       'pre_invoice_discount_pct'],
      dtype='object')

<h2 align="center" style="color:blue">Dropping Redundant Columns</h2>

In [35]:
nmerge_fae.drop(['division','category','product','platform','channel','customer_name'], axis=1, inplace=True)

In [36]:
nmerge_fae.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt'],
      dtype='object')

In [37]:
nmerge_fae.shape

(1858329, 11)

<h2 align="center" style="color:blue">Importing and joining with post_invoice_deductions table </h2>

#### Importing post_invoice_deductions table from post_df.csv

In [39]:
post = pd.read_csv("post_df.csv")
post.shape

(2063076, 5)

In [40]:
post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063076 entries, 0 to 2063075
Data columns (total 5 columns):
 #   Column                Dtype  
---  ------                -----  
 0   customer_code         int64  
 1   product_code          object 
 2   date                  object 
 3   discounts_pct         float64
 4   other_deductions_pct  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 78.7+ MB


In [41]:
post['date'] = pd.to_datetime(post['date'])

#### getting discounts_pct from post df to nmerge_fae

In [43]:
post_merge = pd.merge(
    nmerge_fae,
    post,
    how='left',  # or 'inner', 'right', 'outer' depending on your use case
    on=['customer_code', 'product_code','date']
)

In [44]:
post_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct'],
      dtype='object')

<h2 align="center" style="color:blue">post invoice deductions amount from percentages </h2>

In [45]:
post_merge["post_invoice_dedns_amt"] = post_merge["discounts_pct"] * post_merge["net_invoice_sales_amt"]

In [46]:
post_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt'],
      dtype='object')

In [47]:
post_merge["post_invoice_other_dedns_amt"] = post_merge["other_deductions_pct"] * post_merge["net_invoice_sales_amt"]

In [48]:
post_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Net Sales Amount </h2>

In [49]:
post_merge["net_sales_amt"] = post_merge["net_invoice_sales_amt"] - post_merge["post_invoice_dedns_amt"] - post_merge["post_invoice_other_dedns_amt"]

In [50]:
post_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Importing and joining with manufacturing table </h2>

#### Importing manufacturing table from manufacturing_df.csv

In [52]:
manu = pd.read_csv("manufacturing_df.csv")
manu.shape

(1197, 3)

In [53]:
manu.columns

Index(['product_code', 'cost_year', 'manufacturing_cost'], dtype='object')

In [54]:
manu.rename(columns={'cost_year': 'fiscal_year'}, inplace=True)
manu.columns

Index(['product_code', 'fiscal_year', 'manufacturing_cost'], dtype='object')

#### getting manufacturing_cost from manu df

In [56]:
manu_merge = pd.merge(
    post_merge,
    manu,
    how='left',  # or 'inner', 'right', 'outer' depending on your use case
    on=['product_code', 'fiscal_year']
)

In [57]:
manu_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost'],
      dtype='object')

<h2 align="center" style="color:blue"> Manufacturing Cost </h2>

In [58]:
manu_merge["manufacturing_cost"] = manu_merge["manufacturing_cost"] * manu_merge["qty"]
manu_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost'],
      dtype='object')

<h2 align="center" style="color:blue">Importing and joining with freight table </h2>

#### Importing freight table from freight_df.csv

In [60]:
freight = pd.read_csv("freight_df.csv")
freight.shape

(135, 4)

In [61]:
freight.columns

Index(['market', 'fiscal_year', 'freight_pct', 'other_cost_pct'], dtype='object')

In [62]:
#### getting freight_cost from freight

In [63]:
freight_merge = pd.merge(
    manu_merge,
    freight,
    how='left',  # or 'inner', 'right', 'outer' depending on your use case
    on=['market','fiscal_year']
)

In [64]:
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost',
       'freight_pct', 'other_cost_pct'],
      dtype='object')

<h2 align="center" style="color:blue">Freight cost from percentage </h2>

In [65]:
freight_merge["freight_cost"] = freight_merge["freight_pct"] * freight_merge["net_sales_amt"]
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost',
       'freight_pct', 'other_cost_pct', 'freight_cost'],
      dtype='object')

<h2 align="center" style="color:blue">Other cost from percentage </h2>

In [66]:
freight_merge["other_cost"] = freight_merge["other_cost_pct"] * freight_merge["net_sales_amt"]
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost',
       'freight_pct', 'other_cost_pct', 'freight_cost', 'other_cost'],
      dtype='object')

<h2 align="center" style="color:blue">Total COGS</h2>

In [67]:
freight_merge["total_cogs_amt"] = freight_merge["manufacturing_cost"] + freight_merge["freight_cost"] + freight_merge["other_cost"]

In [68]:
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost',
       'freight_pct', 'other_cost_pct', 'freight_cost', 'other_cost',
       'total_cogs_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Gross Margin Amount</h2>

In [69]:
freight_merge["gross_margin_amt"] = freight_merge['net_sales_amt'] - freight_merge["total_cogs_amt"]

In [70]:
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_price', 'gross_sales_amt', 'pre_invoice_discount_pct',
       'pre_invoice_disc_amt', 'net_invoice_sales_amt', 'discounts_pct',
       'other_deductions_pct', 'post_invoice_dedns_amt',
       'post_invoice_other_dedns_amt', 'net_sales_amt', 'manufacturing_cost',
       'freight_pct', 'other_cost_pct', 'freight_cost', 'other_cost',
       'total_cogs_amt', 'gross_margin_amt'],
      dtype='object')

<h2 align="center" style="color:blue">Dropping unnecessary Columns</h2>

In [71]:
freight_merge.drop(['gross_price','pre_invoice_discount_pct',
                    'discounts_pct','other_deductions_pct','freight_pct','other_cost_pct',], axis=1, inplace=True)

In [72]:
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_sales_amt', 'pre_invoice_disc_amt', 'net_invoice_sales_amt',
       'post_invoice_dedns_amt', 'post_invoice_other_dedns_amt',
       'net_sales_amt', 'manufacturing_cost', 'freight_cost', 'other_cost',
       'total_cogs_amt', 'gross_margin_amt'],
      dtype='object')

freight_merge.drop(['total_cogs_amt', 'gross_margin_amt'], axis=1, inplace=True)

In [74]:
freight_merge.columns

Index(['date', 'product_code', 'market', 'customer_code', 'qty', 'fiscal_year',
       'gross_sales_amt', 'pre_invoice_disc_amt', 'net_invoice_sales_amt',
       'post_invoice_dedns_amt', 'post_invoice_other_dedns_amt',
       'net_sales_amt', 'manufacturing_cost', 'freight_cost', 'other_cost',
       'total_cogs_amt', 'gross_margin_amt'],
      dtype='object')

In [75]:
freight_merge.shape

(1858329, 17)

<h2 align="center" style="color:blue">Final columns in Transaction Data</h2>

In [76]:
freight_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858329 entries, 0 to 1858328
Data columns (total 17 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   date                          datetime64[ns]
 1   product_code                  object        
 2   market                        object        
 3   customer_code                 int64         
 4   qty                           int64         
 5   fiscal_year                   int32         
 6   gross_sales_amt               float64       
 7   pre_invoice_disc_amt          float64       
 8   net_invoice_sales_amt         float64       
 9   post_invoice_dedns_amt        float64       
 10  post_invoice_other_dedns_amt  float64       
 11  net_sales_amt                 float64       
 12  manufacturing_cost            float64       
 13  freight_cost                  float64       
 14  other_cost                    float64       
 15  total_cogs_amt                fl

<h2 align="center" style="color:blue">Optimizing Datatypes</h2>

In [141]:
freight_merge['market'] = freight_merge['market'].astype('category')
freight_merge['product_code'] = freight_merge['product_code'].astype('category')
freight_merge['customer_code'] = freight_merge['customer_code'].astype('category')
freight_merge['fiscal_year'] = freight_merge['fiscal_year'].astype('category')

In [145]:
float_cols = ['gross_sales_amt', 'pre_invoice_disc_amt', 'net_invoice_sales_amt',
              'post_invoice_dedns_amt', 'post_invoice_other_dedns_amt', 'net_sales_amt',
              'manufacturing_cost', 'freight_cost', 'other_cost', 'total_cogs_amt',
              'gross_margin_amt']
freight_merge[float_cols] = freight_merge[float_cols].astype('float32')

In [147]:
freight_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858329 entries, 0 to 1858328
Data columns (total 17 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   date                          datetime64[ns]
 1   product_code                  category      
 2   market                        category      
 3   customer_code                 category      
 4   qty                           int64         
 5   fiscal_year                   category      
 6   gross_sales_amt               float32       
 7   pre_invoice_disc_amt          float32       
 8   net_invoice_sales_amt         float32       
 9   post_invoice_dedns_amt        float32       
 10  post_invoice_other_dedns_amt  float32       
 11  net_sales_amt                 float32       
 12  manufacturing_cost            float32       
 13  freight_cost                  float32       
 14  other_cost                    float32       
 15  total_cogs_amt                fl

In [149]:
freight_merge.to_csv("final_hw.csv", index=False)

In [3]:
fhw = pd.read_csv("final_hw.csv")
fhw.shape

(1858329, 17)

In [5]:
fhw.head()

,date,product_code,market,customer_code,qty,fiscal_year,gross_sales_amt,pre_invoice_disc_amt,net_invoice_sales_amt,post_invoice_dedns_amt,post_invoice_other_dedns_amt,net_sales_amt,manufacturing_cost,freight_cost,other_cost,total_cogs_amt,gross_margin_amt
0,2017-09-01,A6218160101,Australia,70008169,81,2018,1069.2730,101.95376,967.31915,163.93845,101.42222,701.95850,315.2277,13.196819,3.509792,331.93433,370.02417
1,2017-09-01,A6218160101,Australia,90008165,157,2018,2072.5413,590.42220,1482.11910,264.21942,173.24141,1044.65830,610.9969,19.639576,5.223291,635.85974,408.79852
2,2017-09-01,A6218160101,Australia,90008166,126,2018,1663.3134,479.66418,1183.64920,248.15797,133.24884,802.24240,490.3542,15.082157,4.011212,509.44757,292.79483
3,2017-09-01,A6218160101,Australia,90008167,160,2018,2112.1440,411.22906,1700.91490,344.45093,222.56755,1133.89650,622.6720,21.317253,5.669482,649.65875,484.23773
4,2017-09-01,A6218160101,Australia,70008170,120,2018,1584.1080,300.41412,1283.69380,247.07115,153.45836,883.16437,467.0040,16.603490,4.415822,488.02332,395.14105
